In [1]:
from torchvision.io import read_image
import os

path = 'C:/Users/phili/OneDrive/Uni/WS_22/Masterarbeit/Masterarbeit_Code_Philipp_Rosin/Data_set_BCI_challange/train/HE_imgs/HE'
img_name = '00000_train_1+.png'
HE_img_path = os.path.join(path, img_name)
HE_img = read_image(HE_img_path)

In [15]:


kernel_size, stride = 1024,1024
patches = HE_img .unfold(1, kernel_size, stride).unfold(2, kernel_size, stride)
patches = patches.contiguous().view(patches.size(0), -1, kernel_size, kernel_size)
print(patches.shape) # channels, patches, kernel_size, kernel_size


torch.Size([3, 1, 1024, 1024])


In [13]:
import torch
class ConvBlock(torch.nn.Module):
    def __init__(self,input_size,output_size,kernel_size=3,stride=2,padding=1,activation='relu',batch_norm=True):
        super(ConvBlock,self).__init__()
        self.conv = torch.nn.Conv2d(input_size,output_size,kernel_size,stride,padding)
        self.batch_norm = batch_norm
        self.bn = torch.nn.InstanceNorm2d(output_size)
        self.activation = activation
        self.relu = torch.nn.ReLU(True)
        self.lrelu = torch.nn.LeakyReLU(0.2,True)
        self.tanh = torch.nn.Tanh()
    def forward(self,x):
        if self.batch_norm:
            out = self.bn(self.conv(x))
        else:
            out = self.conv(x)
        
        if self.activation == 'relu':
            return self.relu(out)
        elif self.activation == 'lrelu':
            return self.lrelu(out)
        elif self.activation == 'tanh':
            return self.tanh(out)
        elif self.activation == 'no_act':
            return out
            
class DeconvBlock(torch.nn.Module):
    def __init__(self,input_size,output_size,kernel_size=3,stride=2,padding=1,output_padding=0,activation='relu',batch_norm=False):
        super(DeconvBlock,self).__init__()
        self.deconv = torch.nn.ConvTranspose2d(input_size,output_size,kernel_size,stride,padding,output_padding)
        self.batch_norm = batch_norm
        self.bn = torch.nn.InstanceNorm2d(output_size)
        self.activation = activation
        self.   relu = torch.nn.ReLU(True)

    def forward(self,x):
        if self.batch_norm:
            out = self.bn(self.deconv(x))
        else:
            out = self.deconv(x)
        if self.activation == 'relu':
            return self.relu(out)
        elif self.activation == 'lrelu':
            return self.lrelu(out)
        elif self.activation == 'tanh':
            return self.tanh(out)
        elif self.activation == 'no_act':
            return out


In [24]:
# H_out ​= (H_in​−1)*stride[0] − 2×padding[0] + dilation[0]×(kernel_size[0]−1) + output_padding[0] + 1

input = torch.rand([1,3,100,100])

block1 =  torch.nn.Conv2d(3,3,kernel_size=3,stride=1,padding=1)
output = block1(input)
print(input.shape)
print(output.shape)


torch.Size([1, 3, 100, 100])
torch.Size([1, 3, 100, 100])


In [ ]:
class Generator(torch.nn.Module):
    def __init__(self,input_channels,step_num,num_filter,kernel_size):
        super(Generator,self).__init__()
        
        #Reflection padding
        if kernel_size == 7:
            self.padding = torch.nn.ReflectionPad2d(3)
        elif kernel_size == 3:
            self.padding = torch.nn.ReflectionPad2d(1)
        else:
            print('KERNEL_SIZE SHOULD BE EITHER 3 OR 7 FOR THE GENERATORMODELS')

            
        for i in range(step_um):
            self.deconv = torch.nn.ConvTranspose2d(input_channels,output_chan,kernel_size,stride,padding,output_padding)